# Creating future scenarios of crop allocation

This notebook helps creating future scenarios of crop allocation based on a number o factors

**Original code:** [Neeraj Baruah](neeraj.baruah@vivideconomics.com) <br />
**Updates, Modifications:** [Alexandros Korkovelos](https://github.com/akorkovelos) <br />
**Funding:** The World Bank (contract number: 7190531)

In [1]:
# Importing necessary modules

import os, time
import glob
from osgeo import gdal, ogr, osr
from osgeo.gdalnumeric import *
from osgeo.gdalconst import *
import numpy as np
import rasterio
from rasterio.mask import mask
import fiona
import os

### Paths & Directories

In [2]:
# Define path of directories

# This is the original directory containing the main input datasets
dirname = r"N:\Agrodem\Future_Scenarios\Input_data"

# These are derivative directories where intermediate data is stored
TEMP_PATH = os.path.join(dirname, 'temp')
INTERMEDIATE_PATH = os.path.join(dirname, 'intermediate')
OUT_PATH = os.path.join(dirname, 'output')

#### Note that this analysis requires the following datasets/parameters to run:

Raster layers

* Input raster layer(s) for suitability - categorical (see input_cat_lyrs)

* Input raster layer(s) for suitability - continuous (see input_thr_lyrs)

* Input raster yield layers for modelled crop (see input_yield_lyrs)

* Input raster indicating production prices (see input_prod_prices)

* Input World Bank output price factor (see input_wbp)

Vector layers

* Input administrative bound file (see input_bnd)

* Vector layer for location of market areas (see input_markets)

Other parameters

* Input csv for crop output prices (see input_crop_out_prices)

* Input lookup table for categorical layers (see input_lut_list)
   
* Input condition list for continuous layers (see input_con_list)

* Input for attainable yield factor as fraction (see par_ayf)

* Input for crop prices shock (see par_cps)

* Output prices distance decay - Alpha and gamma controls (see o_alpha, o_gamma)

* Production prices distance decay - Alpha and gamma controls  (see p_alpha)

* Future demand for agricultural area (in ha) for certain crop (input_ag_area)

### General Inputs

In [3]:
# Future demand for agricultural area (in ha) for certain crop (maize, cassava, ...)
input_ag_area = [231990] 

# Scenario label
sce_nme = ['maize_SG']

# Input for attainable yield factor as fraction 
par_ayf = [1]  

# Input for crop prices shock     
par_cps = [1]       

# Output prices distance decay - Alpha and gamma controls
o_alpha = [10000]   
o_gamma = [0.2]

# Production prices distance decay - Alpha controls 
p_alpha = [0]       

# Scenario name
sce_nme = '_' + sce_nme[0] + '_{0}_{1}_{2}_{3}_{4}_{5}'.format(str(par_ayf[0]).replace('.', '-'), 
                                                               str(par_cps[0]).replace('.', '-'), 
                                                               str(o_alpha[0]).replace('.', '-'), 
                                                               str(o_gamma[0]).replace('.', '-'), 
                                                               str(p_alpha[0]).replace('.', '-'), 
                                                               str(input_ag_area[0]))

### Processing 

For each process, the relevant function(s) is imported, then input data is set and finally the process executes

#### A. Create binary layers for suitability

In [4]:
def getRaster(directory, rst_nme, band=1, asArray=1):
    
    '''This function returns band or array output from raster file name
    
    INPOUT
    directory: full path to raster file location
    rst_name: full name of the raster file 
    band: band to be used, set to 1 by default
    asArray: set to 1 by default
    
    OUTPUT
    '''
    
    if isinstance(rst_nme, np.ndarray):
        arr = rst_nme
    else:        
        rst = gdal.Open(directory + "//" + rst_nme)
        band_data = rst.GetRasterBand(band)
        arr = BandReadAsArray(band_data)     
        
    if asArray == 1:
        
        return arr
    else:
        return band
    
#########################################################################################################################

def arr2rst(location, ref_rst, arr, output_file, file_type=gdal.GDT_Byte):
    ''' 
    Converts an array to raster
    
    INPUT
    location: full path to raster file location
    ref_rst: full name of the raster file 
    arr: array to be converted
    output_file: full output path
    output_file: type of file
    
    OUTPUT
    '''
    
    ref = gdal.Open(location + "//" + ref_rst, GA_ReadOnly)
    band = ref.GetRasterBand(1)
    proj = ref.GetProjection()
    geotransform = ref.GetGeoTransform()
    xsize = band.XSize
    ysize = band.YSize
    
    gtiff = gdal.GetDriverByName('GTiff') 
    out = gtiff.Create(output_file, xsize, ysize, 1, file_type)
    out.SetProjection(proj)
    out.SetGeoTransform(geotransform)
    
    
    out.GetRasterBand(1).WriteArray(arr) 
    out.FlushCache()
    out = None
    
#########################################################################################################################
    
def sf(outname, path='i'):
    '''
    Function that helps define support directories 
    
    INPUT
    outname: name of the output directory
    '''
    
    if path == 'i':
        return os.path.join(INTERMEDIATE_PATH, outname)
    elif path == 't':
        return os.path.join(TEMP_PATH, outname)
    elif path == 'o':
        return os.path.join(OUT_PATH, outname)
    elif path == 'd':
        return os.path.join(DATA_PATH, outname)
    else:
        print("Please enter folder path as one of below: i - intermediate, t - temp, o - output, d - data, r - rasterpath")

#########################################################################################################################

def reclassify_rasters(raster_list, lookup_tbl_list, out_folder = 'i'):
    '''
    Reclassify a raster numpy array using a look-up table in the form of a tuple-list; only for categorical values
    
    INPUT
    raster_list: list containing full name of rasters to be re-classified
    lookup_tbl_list: table to be used for re-classification; format should be like [(old_value_1, new_value_1), (old_value_2, new_value_2), ...] 
    out_folder: string that indicates where the output will be stored (see sf function)
    
    OUTPUT
    Re-classified raster layers
    '''
    #lookup_tbl = 
    ### May need to clip
    
    counter = 0
    for r in raster_list:
        print("Reclassifying", r)

        rst_nme = r.split('.')[0] + '_sbin.tif'  ## to change
        rst_arr = getRaster(dirname, r, band=1, asArray=1)
        rst_arr = rst_arr.astype(np.uint16)
        
        idx, val = np.asarray(lookup_tbl_list[counter]).T
        lookup_array = np.zeros(max(idx) + 1)

        lookup_array[idx] = val

        new_array = lookup_array[rst_arr]

        arr2rst(dirname, r, new_array, sf(rst_nme, out_folder), file_type=gdal.GDT_UInt16)
        counter = counter + 1
    print ("Completed")
        
#########################################################################################################################

def threshold_rasters(location, raster_list, condition_list, outnme='t'):
    '''
    Threshold rasters based on conditions; Condition list of the form - ['>::12', '<=::3']
    
    INPUT
    location: full path to raster file location
    raster_list: list containing full name of rasters to be thresholded
    condition_list: contains conditions to be used for thresholding; format should be like e.g. ['<=::25']
    
    OUTPUT
    Thresholded raster(s)
    '''
    ## May need to clip
    
    counter = 0
    for r in raster_list:
        print ("Thresholding", r)

        rst_nme = r.split('.')[0] + '_sbin.tif'  
        rst_arr = getRaster(location, r, band=1, asArray=1)
        
        cond = condition_list[counter].split('::')[0]
        thresh = float(condition_list[counter].split('::')[1])
        
        if cond == '<':
            rst = np.where(rst_arr < thresh, 1, 0)
        elif cond == '>':
            rst = np.where(rst_arr > thresh, 1, 0)
        elif cond == '<=':
            rst = np.where(rst_arr <= thresh, 1, 0)
        elif cond == '>=':
            rst = np.where(rst_arr >= thresh, 1, 0)
        elif cond == '==':
            rst = np.where(rst_arr == thresh, 1, 0)
        elif cond == '!=':
            rst = np.where(rst_arr != thresh, 1, 0)
        else:
            print("Only >, <, >=, <=, == and != are allowed!")
        
        arr2rst(location, r, rst, sf(rst_nme, outnme), file_type=gdal.GDT_Byte)
        counter = counter + 1
    print ("Completed")     

Input data

In [5]:
# Input layers for suitability which are categorical (to be reclassified)
input_cat_lyrs = ['moz_landuse_500_clip.tif', 'moz_pas_500_clip.tif', 'moz_flood_rp50_clip_500.tif'] 

# Input lookup table for categorical layers ## Pre-process
input_lut_list = [[(1,1), (2,1), (3,1), (4,0), (5,1), (6,1), (7,1), (8,0), (10,0), (15, 0)], [(0,1), (1, 0), (3,0)], [(0,1), (1,0)]]      ## Forest cover allowed

# Input condition list for continuous layers   ## Pre-process
input_con_list = ['<=::25']

# Input layers for suitability which are continuous (to be thresholded) ## Spatial Layers(s) 
input_thr_lyrs = ['moz_pop10_500_clip.tif']

Execution

In [6]:
# reclasify rasters
reclassify_rasters(input_cat_lyrs, lookup_tbl_list=input_lut_list, out_folder = 't') 

# Adding threshold to rasters
threshold_rasters(dirname, input_thr_lyrs, input_con_list)

Reclassifying moz_landuse_500_clip.tif
Reclassifying moz_pas_500_clip.tif
Reclassifying moz_flood_rp50_clip_500.tif
Completed
Thresholding moz_pop10_500_clip.tif
Completed


#### B. Create suitability layer

In [7]:
def iterateRasters(folder_nme, wildcard):
    '''
    Iterate rasters in a folder and get a list
    
    INPUT
    folder_nme: full name of derectory where rasters are stored
    wildcard:
    
    OUTPUT
    List of output rasters
    '''
    pathlist = glob.glob(os.path.join(folder_nme, wildcard))
    
    paths = []
    for path in pathlist:         
         path_in_str = str(path)
         paths.append(path_in_str)
    return paths
        
#########################################################################################################################

def multiply_binary_rasters(rasters, outnme):
    '''
    Multiply binary rasters
    
    INPUT
    rasters: list of raster to be multiplied
    
    OUTPUT 
    Result raster product of multiplication
    '''
    print ("Preparing suitability layer...")
    rst = getRaster(TEMP_PATH, rasters[0].split(TEMP_PATH + "\\")[1])
    
    for r in rasters[1:]:
        ar = getRaster(TEMP_PATH, r.split(TEMP_PATH + "\\")[1])
        rst = rst*ar
        
    arr2rst(TEMP_PATH, rasters[0].split(TEMP_PATH + "\\")[1], rst, outnme, file_type=gdal.GDT_Byte)
    print ("Completed")

Execution

In [8]:
suitability_file = sf('suitability{x}.tif'.format(x=sce_nme), 'i')
multiply_binary_rasters(iterateRasters(TEMP_PATH, '*_sbin.tif'), suitability_file) 

Preparing suitability layer...
Completed


#### C. Create yield layer

In [9]:
def weighted_sum(raster_list, weights_list, outnme):
    '''
    Create weighted sum of rasters
    
    INPUT
    raster_list: list of raster files for which the weighted sum will be calculated
    weights_list: list of weights to be used
    outnme: name of the output file
    
    OUTPUT
    Weighted sum raster file
    '''
    print ("Creating a weighted sum of rasters...")
    rst = getRaster(dirname, raster_list[0]) * weights_list[0]
    
    if len(raster_list) > 1:
        counter = 1
        for r in raster_list[1:]:
            ar = getRaster(dirname, r)
            rst = rst + (ar * weights_list[counter])
            counter = counter + 1
    
    arr2rst(dirname, raster_list[0], rst, outnme, file_type=gdal.GDT_Float32)
    print ("Complete")

Input data

In [10]:
# Input yield layers for major crops   ## Spatial Layers(s)
input_yield_lyrs = ['mz_rf_low_500_clip_utm.tif'] 

# Input for attainable yield factor as fraction  ## Core param 
par_ayf = [1] 

Execution

In [11]:
yield_file = sf('yield{x}.tif' .format(x=sce_nme), 'i')
weighted_sum(input_yield_lyrs, par_ayf, yield_file)

Creating a weighted sum of rasters...
Complete


#### D. Create distance from markets

In [12]:
def distRasters(location, rst, outnme):
    '''
    Calculate distance to markets
    
    INPUT
    location: full path of directory containing rasters
    rst: raster to be used
    outnme: full path of output directory
    
    OUTPUT
    Raster indicating distance to markets
    '''
    print ("Calculating distance to markets...")
    ref = gdal.Open(location + "//" + rst, GA_ReadOnly)
    band = ref.GetRasterBand(1)
    proj = ref.GetProjection()
    geotransform = ref.GetGeoTransform()
    xsize = band.XSize
    ysize = band.YSize
    
    gtiff = gdal.GetDriverByName('GTiff')
    DataSet = gtiff.Create(outnme, xsize, ysize, 1, gdal.GDT_UInt16)
    DataSet.SetGeoTransform(geotransform)
    DataSet.SetProjection(proj)
    
    options = []
    
    gdal.ComputeProximity(band, DataSet.GetRasterBand(1), options, callback = None)
    print ("Completed")

Input data

In [13]:
# Input market areas as points and tiff     ## Spatial Layer(s)
input_markets = ['moz_markets.shp', 'moz_markets_clip.tif', 'moz_dist2markets_500_clip.tif'] 

Execution

In [14]:
mar_dist = sf('dist_2_markets{x}.tif' .format(x=sce_nme), 'o')
distRasters(dirname, input_markets[1], mar_dist)

Calculating distance to markets...
Completed


#### D. Get farmgate output prices and production costs for revenue

In [15]:
def o_distance_decay(prices_file, dist_file, alpha, gamma, shock_factor, output):
    '''
    Distance decay equation (negative exp)
    '''
    print ("Calculating negative exp of d.decay equation...")
    distance_raster = getRaster(OUT_PATH, dist_file, band=1, asArray=1)
    output_price = getRaster(dirname, prices_file, band=1, asArray=1)
    output_price = output_price * shock_factor
    
    base_decay = 1.0 - (np.exp((-alpha/distance_raster))*gamma)
    arr2rst(dirname, prices_file, base_decay, 'basedecay_test.tif', file_type=gdal.GDT_Float32)
    o_decay = output_price * base_decay
    arr2rst(dirname, prices_file, o_decay, output, file_type=gdal.GDT_Float32)
    print("Completed")
    
#########################################################################################################################

def p_distance_decay(costs_file, dist_file, alpha, suppress, output):
    '''
    Distance decay equation (positive exp)
    '''
    print("Calculating positive exp of d.decay equation...")
    distance_raster = getRaster(OUT_PATH, dist_file)
    cost_price = getRaster(dirname, costs_file)
    cost_price = cost_price * suppress
    
    base_decay = np.exp(alpha * distance_raster)
    p_decay = cost_price * base_decay
    arr2rst(dirname, costs_file, p_decay, output, file_type=gdal.GDT_Float32)
    print("Completed")

Input data

In [16]:
# Input csv for crop output prices  ## Data Layer(s)
input_crop_out_prices = ['mz_output_price_per_ha.tif']

mar_dist = "dist_2_markets_maize_SG_1_1_10000_0-2_0_231990.tif"
#mar_dist = input_markets[2]

# Input production prices      ## Spatial Layer(s)
input_prod_prices = ['mz_cost_price_per_ha.tif']                                                     

# Input WB output price factor  ## Sensitivity param *
input_wbp = [1]

Execution

Negative exponential of the distance decay equation

In [17]:
output_prices = sf('output_prices{x}.tif' .format(x=sce_nme), 'i')
o_distance_decay(input_crop_out_prices[0], mar_dist, o_alpha[0], o_gamma[0], par_cps[0], output_prices)

Calculating negative exp of d.decay equation...


C:\Users\adm.esa\Anaconda3\envs\OSMNX\lib\site-packages\ipykernel_launcher.py:10: RuntimeWarning: divide by zero encountered in true_divide
  # Remove the CWD from sys.path while we load stuff.


Completed


Positive exponential of the distance decay equation

In [18]:
cost_prices = sf('cost_prices{x}.tif' .format(x=sce_nme), 'i') 
p_distance_decay(input_prod_prices[0], mar_dist, p_alpha[0], input_wbp[0], cost_prices)

Calculating positive exp of d.decay equation...
Completed


#### E. Get gross revenue and profitability 

In [19]:
def array_calc(a,b, output, calc = '-', filetype = gdal.GDT_Float32):
    '''
    Array calculator
    '''
    print ("Calculating array multiplication")
    a_ar = getRaster(INTERMEDIATE_PATH, a)
    b_ar = getRaster(INTERMEDIATE_PATH, b)   
    
    if calc == '*':
        f = a_ar * b_ar
    elif calc == '+':
        f = a_ar + b_ar
    elif calc == '-':
        f = a_ar - b_ar
    elif calc == '/':
        f = a_ar/b_ar
    else:
        print("Please enter calc as one of: *, +, -, /")
      
    arr2rst(INTERMEDIATE_PATH, a, f, output, file_type=filetype)
    print ("Completed")

Input data

In [20]:
# Input files
yield_file_name = yield_file.split(INTERMEDIATE_PATH + "\\")[1] 
output_prices_name = output_prices.split(INTERMEDIATE_PATH + "\\")[1] 

Execution

Gross revenue

In [21]:
gross_revenue = sf('gross_revenue{x}.tif'.format(x=sce_nme), 'i') 
array_calc(yield_file_name, output_prices_name, gross_revenue, calc = '*')

Calculating array multiplication
Completed


Revenue

In [22]:
# Input files
gross_revenue_name = gross_revenue.split(INTERMEDIATE_PATH + "\\")[1]
cost_prices_name = cost_prices.split(INTERMEDIATE_PATH + "\\")[1]

In [23]:
revenue = sf('revenue{x}.tif'.format(x=sce_nme), 'i') 
array_calc(gross_revenue_name, cost_prices_name, revenue, calc = '-')

Calculating array multiplication
Completed


Final Revenue (including suitability)

In [24]:
# Input files
suitability_file_name = suitability_file.split(INTERMEDIATE_PATH + "\\")[1]
revenue_name = revenue.split(INTERMEDIATE_PATH + "\\")[1]

In [25]:
f_revenue = sf('final_revenue{x}.tif'.format(x=sce_nme), 'i') 
array_calc(suitability_file_name, revenue_name, f_revenue, calc = '*')

Calculating array multiplication
Completed


#### F. Final raster clip

In [26]:
def raster_clip(Input_raster_path,vector_clip_path, outnme):
    """
     This function clips a raster file with a provided polygon and writes the 
     result to a geotiff file
    
    Inputs
    Input_raster_path - Full file path to raster file 
    vector_clip_path - Full file path to shapefile to be used in clipping the raster file
    """
    print ("Clipping raster file to boundaries...")
    with fiona.open(vector_clip_path) as shapefile:
        geoms = [feature["geometry"] for feature in shapefile]
    with rasterio.open(Input_raster_path) as src:
        out_image, out_transform = mask(src, geoms, crop=True)
    out_meta = src.meta.copy()
    
    # save the resulting raster  
    out_meta.update({"driver": "GTiff",
        "height": out_image.shape[1],
        "width": out_image.shape[2],
    "transform": out_transform})
    
    with rasterio.open(outnme, "w", **out_meta) as dest:
        dest.write(out_image)
    print ("Completed")

Input data

In [27]:
# Input administrative bound file  ## Data Layer(s)
input_bnd = dirname + "\\" + 'moz_bnd.shp'

Execution

In [28]:
final_layer = sf('profitability{x}.tif' .format(x=sce_nme), 'i') 
raster_clip(f_revenue, input_bnd, final_layer)

Clipping raster file to boundaries...
Completed


#### G. Get binary cropland values (threshold)

In [29]:
final_layer_name = final_layer.split(INTERMEDIATE_PATH + "\\")[1] 

In [30]:
threshold_rasters(INTERMEDIATE_PATH, [final_layer_name], condition_list=['>::0']) 

Thresholding profitability_maize_SG_1_1_10000_0-2_0_231990.tif
Completed


In [31]:
def get_threshold(potential_raster, pixels):
    '''
    Convert potential raster to urban forecasts using number of pixels to convert
    '''
    print ("Converting potential extent to urban forcast...")
    urb_pot = getRaster(INTERMEDIATE_PATH, potential_raster)    
    urb_pot = urb_pot[urb_pot > 0]
    
    u,v = np.unique(urb_pot, return_counts=True)
    total_avail_pixels = sum(v)
    thresh = sorted(urb_pot, reverse=True)[:pixels][-1]
        
    return(thresh)

#########################################################################################################################

def area_to_pixels(demand, unit = 'km2', pixel_size = 30):
    '''
    Convert area demand to pixels
    '''
#   demand = demand[0]
    pixel_area = pixel_size * pixel_size
    if unit == 'km2':
        pixels = int(round((demand*1000000.0)/(pixel_area),0))
    elif unit == 'm2':
        pixels = int(round((demand)/(pixel_area),0))
    elif unit == 'ha':
        pixels = int(round((demand*10000.0)/(pixel_area),0))   
    else:
        print("Demand should be in m2, km2 or ha")
        
    return pixels

Execution

In [32]:
threshold = get_threshold(final_layer_name, pixels = area_to_pixels(demand = input_ag_area[0], unit = 'ha', pixel_size = 500))
threshold_rasters(INTERMEDIATE_PATH, [final_layer_name], ['>=::{0}'.format(str(threshold))],'o')

Converting potential extent to urban forcast...
Thresholding profitability_maize_SG_1_1_10000_0-2_0_231990.tif
Completed


#### H. Exporting final raster in output directory and proper naming

In [33]:
def CreateGeoTiff(location, ref_rst, outmame):
    '''
    Convert array to a one band raster
    
    INPUT
    location: full path of your input raster
    output_file: full path and name of the output raster
    ref_rst: input raster
    
    OUTPUT
    Basically renaming a raster with gdal
    ''' 
    print (" Exporting raster layer")
    ref = gdal.Open(location + "\\" + ref_rst, GA_ReadOnly)
    band = ref.GetRasterBand(1)
    proj = ref.GetProjection()
    geotransform = ref.GetGeoTransform()
    xsize = band.XSize
    ysize = band.YSize
    num_bands = 1
    
    
    gtiff = gdal.GetDriverByName('GTiff')
    DataSet = gtiff.Create(outmame, xsize, ysize, num_bands, gdal.GDT_Byte)
    DataSet.SetGeoTransform(geotransform)
    DataSet.SetProjection(proj)
        
    DataSet.FlushCache()
    print ("Task completed")


Input data

In [34]:
fin_layers = iterateRasters(OUT_PATH,'*_sbin.tif')
fin_layers_name = fin_layers[0].split(OUT_PATH + "\\")[1]
final_output = (OUT_PATH + "//" +'Final_Projected_area_for{x}.tif'.format(x=sce_nme)) 

Execution

In [35]:
CreateGeoTiff(OUT_PATH, fin_layers_name, final_output)

 Exporting raster layer
Task completed


## Additional functions that might be useful
(Not included in any of the runs above though)

In [36]:
def resample():
    '''
    Resample
    ''' 
    
    from osgeo import gdal, gdalconst
    
    inputfile = r"C:\Users\Development\Downloads\mini_mozambique.tif"
    inputf = gdal.Open(inputfile)
    inputProj = inputf.GetProjection()
#    inputTrans = inputf.GetGeoTransform()
    
    referencefile = r"C:\Users\Development\Downloads\MODIS_landcover\MCD12Q1.A2010001.h21v10.006.2018146000538.hdf"
    reference1 = gdal.Open(referencefile)
    reference = gdal.Open(reference1.GetSubDatasets()[0][0])
    referenceProj = reference.GetProjection()
    referenceTrans = reference.GetGeoTransform()
#    bandreference = reference.GetRasterBand(1)    
    x = reference.RasterXSize 
    y = reference.RasterYSize
    print(type(x),type(y))
    print(referenceProj)
    print(referenceTrans)
    
    
    outputfile = "1km_downscaled.tif"
    driver= gdal.GetDriverByName('GTiff')
    print(type(outputfile),type(x),type(y))
    output = driver.Create(outputfile,x,y,1,gdal.GDT_Float32)
    output.SetGeoTransform(referenceTrans)
    output.SetProjection(referenceProj)
    
    gdal.ReprojectImage(input,output,inputProj,referenceProj,gdalconst.GRA_Bilinear)

    del output
    
    # TO DO : Resample raster grid using multiple algos - nearest neighbour, average, median, majority.
    # Source: https://gis.stackexchange.com/questions/234022/resampling-a-raster-from-python-without-using-gdalwarp  
    

#########################################################################################################################
      
def rasterise(Shapefile_path,Output_filename):
    '''
    RASTERISE
    '''
    
    # 1) opening the shapefile    
    
    source_ds = ogr.Open(Shapefile_path)
    source_layer = source_ds.GetLayer()
    
    
    # 2) Creating the destination raster data source
    
    pixelWidth = pixelHeight = 0.1 # depending how fine you want your raster ##COMMENT 1
    x_min, x_max, y_min, y_max = source_layer.GetExtent()
    cols = int((x_max - x_min) / pixelHeight)
    rows = int((y_max - y_min) / pixelWidth)
    
    target_ds = gdal.GetDriverByName('GTiff').Create(Output_filename, cols, rows, 1, 
    gdal.GDT_Float32) ##COMMENT 2
    
    target_ds.SetGeoTransform((x_min, pixelWidth, 0, y_max, 0, -pixelHeight) )##COMMENT 3
    
    # 5) Adding a spatial reference ##COMMENT 4
    target_dsSRS = osr.SpatialReference()
    target_dsSRS.ImportFromEPSG(32736)
    target_ds.SetProjection(target_dsSRS.ExportToWkt())
    
    band = target_ds.GetRasterBand(1) 
    band.SetNoDataValue(0) ##COMMENT 5
    
    gdal.RasterizeLayer(target_ds, [1], source_layer,burn_values=[1], options=["ALL_TOUCHED=TRUE"]) ##COMMENT 6
    
    target_ds = None ##COMMENT 7
    
    # TO DO: Convert shapefile of points and polygons into raster (burn 1, or by attribute)
    # Souce: https://gis.stackexchange.com/questions/212795/rasterizing-shapefiles-with-gdal-and-python 
    
#########################################################################################################################

def majority_filter(raster, mask_rst, exclude):
    '''
    Majority filter
    '''
    arr = getRaster(raster, band=1, asArray=1)
    arr = arr.astype(np.float16)
    marr = getRaster(mask_rst, band=1, asArray=1)
    u,v = np.unique(arr, return_counts=True)
    print(u)
    for e in exclude:
        arr[arr == e] = np.nan
    u,v = np.unique(arr, return_counts=True)
    print(u)

#########################################################################################################################

def clipRaster(raster, mask, outnme, filetype=GDT_Float32):
    '''
    Clip raster using a mask
    '''
    arr = getRaster(raster, band=1, asArray=1)
    mask_arr = getRaster(mask, band=1, asArray=1)
    
    arr = arr*mask_arr
    arr2rst(raster, arr, outnme,  file_type=filetype)
    
#########################################################################################################################

def getPixelValue(rst):
    '''
    Get pixel size
    '''
    ds = gdal.Open(rst)
    gt = ds.GetGeoTransform()
    pixelSize = round(gt[1],0)
 
    return pixelSize
